In [35]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
from sklearn.preprocessing import LabelEncoder

In [38]:
data=pd.read_csv("NationalNames.csv")

In [39]:
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [40]:
X,y=data.iloc[:,1].values , data.iloc[:,3].values

In [41]:
y

array(['F', 'F', 'F', ..., 'M', 'M', 'M'], dtype=object)

In [42]:
#y=LabelEncoder().fit_transform(y)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [44]:
cv=CountVectorizer(analyzer='char')

In [45]:
X_vec=cv.fit_transform(X_train).toarray()

In [46]:
X_vec.shape

(1223040, 26)

In [47]:
X_vec_test=cv.transform(X_test).toarray()

In [48]:
X_vec_test.shape

(602393, 26)

In [49]:
cv.inverse_transform(X_vec[0])

[array(['i', 'l', 'w', 'y'], dtype='<U1')]

In [50]:
X_vec[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0], dtype=int64)

In [17]:
mnb=MultinomialNB()

In [51]:
mnb.fit(X_vec,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [52]:
mnb.predict(X_vec_test[:5])

array(['F', 'M', 'M', 'F', 'F'], dtype='<U1')

In [53]:
y_train[:5]

array(['M', 'F', 'M', 'F', 'F'], dtype=object)

In [54]:
mnb.score(X_vec_test,y_test)

0.6680804723826472

In [56]:
name="naman"

name_v=cv.transform([name]).toarray()

#name_v.shape
mnb.predict(name_v)

array(['F'], dtype='<U1')

In [34]:
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [33]:
## Kaggle code

In [48]:
data_k=data.groupby(["Name","Gender"],as_index=False)["Count"].sum()

In [49]:
data_k.head()

,Name,Gender,Count
0,Aaban,M,72
1,Aabha,F,21
2,Aabid,M,5
3,Aabriella,F,10
4,Aadam,M,196


In [74]:
name_chart=data_k.reset_index().pivot('Name', 'Gender', 'Count')
name_chart=name_chart.fillna(0)
name_chart["Mpre"]=(name_chart["M"]-name_chart["F"])/(name_chart["M"]+name_chart["F"])
name_chart["Gender"]= np.where(name_chart["Mpre"]>0.01,"Male","Female")

In [76]:
name_chart.head()

Gender,F,M,Mpre,Gender
Name,,,,
Aaban,0.0,72.0,1.0,Male
Aabha,21.0,0.0,-1.0,Female
Aabid,0.0,5.0,1.0,Male
Aabriella,10.0,0.0,-1.0,Female
Aadam,0.0,196.0,1.0,Male


In [77]:
#name_chart.index

Index(['Aaban', 'Aabha', 'Aabid', 'Aabriella', 'Aadam', 'Aadan', 'Aadarsh',
       'Aaden', 'Aadesh', 'Aadhav',
       ...
       'Zyshon', 'Zyshonne', 'Zytaevius', 'Zytaveon', 'Zytavion', 'Zytavious',
       'Zyvion', 'Zyyanna', 'Zyyon', 'Zzyzx'],
      dtype='object', name='Name', length=93889)

In [254]:
##from scipy.sparse import coo_matrix

In [255]:
char_vec=CountVectorizer(analyzer="char",ngram_range=(2,2))

In [256]:
X=char_vec.fit_transform(name_chart.index)

In [257]:
X=X.toarray()
y = (name_chart.Gender == 'male').values.astype(np.int)

In [258]:
print(X)

[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 1 1]]


In [259]:
itrain, itest = train_test_split(range(name_chart.shape[0]), train_size=0.7)
mask=np.ones(name_chart.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [260]:
mask.shape 

#  ~mask   This will reverse means True means negative and vice versa

(93889,)

In [261]:
mask[-5:]

array([False,  True,  True,  True,  True])

In [262]:
Xtrainthis=X[mask]
Ytrainthis=y[mask]
Xtestthis=X[~mask]
Ytestthis=y[~mask]

clf = MultinomialNB(alpha = 1)
clf.fit(Xtrainthis, Ytrainthis)
training_accuracy = clf.score(Xtrainthis,Ytrainthis)
test_accuracy = clf.score(Xtestthis,Ytestthis)
        
print(training_accuracy)
print(test_accuracy)

1.0
1.0


In [263]:
def star(sr):
    name=str(sr)
    new=char_vec.transform([name])
    y_pred=clf.predict(new)
    if (y_pred >= .5):
        print("This is most likely a male name!")
    else:
        print("This is most likely a female name!")

In [264]:
star("dev")

This is most likely a female name!
